### importing libraries
- pandas: for organizing data
- re: for extracting agent information
- BeautifulSoup: web scraping on vlr.gg
- requests: connecting to vlr.gg
- numpy: calculation

In [1]:
import pandas as pd
import re
""" from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures """
from bs4 import BeautifulSoup
import requests
import numpy as np
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

### Getting the stats page on vlr.gg

In [2]:
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'}
stats_url = 'https://www.vlr.gg/stats'
result = requests.get(stats_url, headers=headers)
src = result.content
soup = BeautifulSoup(src, 'html.parser')

### Finding the data section

In [3]:
table = soup.find_all("table", class_="wf-table mod-stats")[0].find("tbody").find_all("tr")

### Convert the data section to a pandas dataframe

In [4]:
np_arr = []

for row in table:
  arr = []

  # getting the first 5 rows
  name = row.find("div", class_="text-of")
  country = row.find("div", class_="stats-player-country")
  agents = row.find_all("img")
  rnd = row.find("td", class_="mod-rnd")
  stats = row.find_all("div", class_="color-sq")

  arr.append(name.contents[0])
  arr.append(None if country == None or len(country.contents) == 0 else country.contents[0])

  # getting an array of agents
  agent_regex = re.compile(r'\/img\/vlr\/game\/agents\/([a-z]+)\.png')
  agent_arr = []
  for agent in agents:
    src = agent['src']
    m = re.search(agent_regex, src)
    if m != None:
      agent_arr.append(m.group(1))
  arr.append(agent_arr)

  arr.append(rnd.contents[0])

  # col ACS to CL%
  for data in stats:
    span = data.find("span")
    arr.append(np.NAN if span == None or len(span.contents) == 0 else span.contents[0])

  # col CL to FD
  cl = row.find("td", class_="mod-cl")
  kmax = row.find("td", class_="mod-a mod-kmax")['data-sort-value']
  rest = row.find_all("td", class_=None)
  rest_data = []
  for data in rest:
    rest_data.append(data.contents[0])
  arr.append(cl.contents[0].replace('\n', '').replace('\t', ''))
  arr.append(kmax)
  arr += rest_data

  np_arr.append(arr)
# convert to an np array
arr = np.array(np_arr, dtype=object)

### Creating a pandas dataframe

In [5]:
df = pd.DataFrame(arr, columns=['player', 'country', 'agents', 'RND', 'ACS', 'K:D', 
'ADR', 'KPR', 'APR', 'FKPR', 'FDPR', 'HS%', 'CL%', 'CL', 'KMAX', 'K', 'D', 'A', 'FK', 'FD'])
df

,player,country,agents,RND,ACS,K:D,ADR,KPR,APR,FKPR,FDPR,HS%,CL%,CL,KMAX,K,D,A,FK,FD
0,SkRossi,GE,[jett],719,287.1,1.46,176.7,1.00,0.14,0.26,0.15,25%,18%,9/51,42,719,491,103,187,105
1,wronski,PEAC,"[jett, reyna]",660,282.4,1.25,175.7,0.97,0.18,0.26,0.18,32%,11%,5/44,37,643,513,121,174,119
2,f0rsakeN,PRX,"[jett, skye]",651,281.3,1.33,170.7,0.98,0.24,0.22,0.15,27%,18%,7/40,30,636,477,155,146,96
3,Sacy,VKS,"[jett, raze]",328,279.7,1.40,170.9,0.98,0.27,0.19,0.10,23%,18%,5/28,30,320,228,87,61,34
4,Supmah,HSDI,"[astra, viper]",315,273.4,1.30,171.1,0.91,0.35,0.19,0.13,24%,13%,2/16,31,287,220,111,61,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,Harmon,DZ,"[sova, sage]",330,154.1,0.69,111.0,0.49,0.34,0.07,0.06,24%,6%,3/48,18,161,232,112,22,20
420,crow,ZETA,"[sage, breach]",392,153.8,0.84,108.1,0.52,0.40,0.04,0.04,17%,7%,3/45,15,205,243,157,17,17
421,Jonaaa6,SS,"[astra, omen]",820,150.0,0.85,100.7,0.55,0.34,0.03,0.05,23%,19%,21/113,25,454,534,280,28,39
422,tdawgg,Zeal,"[viper, omen]",326,149.8,0.65,104.2,0.51,0.36,0.07,0.11,19%,9%,4/46,14,166,256,117,22,35
